<a href="https://colab.research.google.com/github/HuyenNguyenHelen/CORD-19-KG/blob/master/QA-models/covid_19_kg_mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preliminaries

In [1]:
# If using Google Colab run this cell 

# select tensorflow version for colab 
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import tensorflow as tf 

print('TensorFlow  version: {}'.format(tf.__version__))

# Get the GPU name
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

TensorFlow  version: 1.15.2
Found GPU at: /device:GPU:0


In [3]:
%%capture 
# Install AmpliGraph library
! pip install ampligraph

# Required to visualize embeddings with tensorboard projector, comment out if not required!
! pip install --user tensorboard

# Required to plot text on embedding clusters, comment out if not required!
! pip install --user git+https://github.com/Phlya/adjustText

In [4]:
# All imports used in this tutorial 
%tensorflow_version 1.x
import ampligraph
import numpy as np
import pandas as pd
import tensorflow as tf
from ampligraph.datasets import load_fb15k_237
from ampligraph.evaluation import train_test_split_no_unseen, evaluate_performance, mr_score, mrr_score, hits_at_n_score
from ampligraph.discovery import query_topn, discover_facts, find_clusters
from ampligraph.latent_features import TransE, ComplEx, HolE, DistMult, ConvE, ConvKB
from ampligraph.utils import save_model, restore_model

def display_aggregate_metrics(ranks):
    print('Mean Rank:', mr_score(ranks)) 
    print('Mean Reciprocal Rank:', mrr_score(ranks)) 
    print('Hits@1:', hits_at_n_score(ranks, 1))
    print('Hits@10:', hits_at_n_score(ranks, 10))
    print('Hits@100:', hits_at_n_score(ranks, 100))

print('Ampligraph version: {}'.format(ampligraph.__version__))

Ampligraph version: 1.4.0


In [5]:
from ampligraph.datasets import load_fb15k_237, load_wn18rr, load_yago3_10
import pandas as pd
import requests
import io

## Loading Data 
(a triples dataset)

In [6]:
# Downloading the csv file from your GitHub account
# re_pdf = requests.get('https://raw.githubusercontent.com/HuyenNguyenHelen/CORD-19-KG/master/Data/new_triples_with_predefined_relations_pdf_June20_.csv').content
# re_pmc = requests.get('https://raw.githubusercontent.com/HuyenNguyenHelen/CORD-19-KG/master/Data/new_triples_with_predefined_relations_pmc_June20_.csv').content
# pdf = pd.read_csv(io.StringIO(re_pdf.decode('utf-8')))
# pmc =  pd.read_csv(io.StringIO(re_pmc.decode('utf-8')))
# dataset = pd.concat([pmc, pdf])

re_dataset = requests.get('https://raw.githubusercontent.com/HuyenNguyenHelen/CORD-19-KG/master/Data/all-final-cleaned-triples_2.csv').content
dataset = pd.read_csv(io.StringIO(re_dataset.decode('utf-8')))

print(dataset.columns)
dataset.drop(columns='Unnamed: 0', inplace=True)
# dataset = dataset[['subject','new_relation', 'object' ]]
dataset.head(5)

Index(['Unnamed: 0', 'subject', 'new_relation', 'object'], dtype='object')


,subject,new_relation,object
0,dysfunction,disease_species,child
1,et dysfunction,disease_species,child
2,rsv,disease_disease,virus
3,treatment,treat_procedure_species,child
4,urgency,disease_disease,virus


In [7]:
print('Total triples in the KG:', dataset.shape)

Total triples in the KG: (86275, 3)


In [8]:
print('the number of relations: \n', dataset['new_relation'].value_counts())

the number of relations: 
 disease_disease                    22991
gene_gene                          10985
gene_disease                        6347
disease_species                     5956
disease_gene                        5684
treat_procedure_disease             3757
disease_treat_procedure             3471
gene_treat_procedure                2618
drug_disease                        2420
treat_procedure_gene                2287
gene_species                        2140
treat_procedure_treat_procedure     2115
drug_gene                           2040
gene_drug                           1539
drug_drug                           1505
disease_symptom                     1475
disease_drug                        1407
symptom_disease                     1296
treat_procedure_species             1214
drug_treat_procedure                1124
treat_procedure_drug                1094
drug_species                         892
symptom_symptom                      528
gene_symptom                  

## Data exploration

# Data Processing
Partition the triples into 10 portions that will be used for buiding QA systems

In [9]:
# Shuffle the dataset
dataset = dataset.sample(frac=1, axis=1).reset_index(drop=True)
subsets = np.array_split(dataset, 10)  
len(subsets)


10

# Developing QA system

## Developing model

### Create training, validation and test splits

In [10]:
from ampligraph.evaluation import train_test_split_no_unseen
def train_set_split (ds, test_frac = 200, val_frac = 100 ):
  # get the validation set of size 500
  test_train, X_test = train_test_split_no_unseen(ds.values, test_frac, seed=0)

  # get the test set of size 1000 from the remaining triples
  X_train, X_val = train_test_split_no_unseen(test_train, val_frac, seed=0)

  print('Total triples:', dataset.shape)
  print('Size of train:', X_train.shape)
  print('Size of valid:', X_val.shape)
  print('Size of test:', X_test.shape)
  return X_train, X_val, X_test

### Model Training

In [ ]:
from ampligraph.latent_features import TransE

def training (train_ds, val_ds, test_ds):
  # Train a KGE model
  model = TransE(k=300, 
                epochs=100, 
                eta=1, 
                loss='multiclass_nll', 
                initializer='xavier', initializer_params={'uniform': False},
                regularizer='LP', regularizer_params= {'lambda': 0.001, 'p': 3},
                optimizer= 'adam', optimizer_params= {'lr': 0.0001}, 
                seed= 0, batches_count= 100, verbose=True)

  model.fit(train_ds)
  # ----------------------
  # Evaluate: 
  # Filtered evaluation with ranking strategy assigning worst rank to break ties
  from ampligraph.utils import save_model, restore_model
  save_model(model, 'TransE.pkl')
  model = restore_model('TransE.pkl')

  # create the filter 
  X_filter = np.concatenate([train_ds, val_ds, test_ds], 0)

  # compute ranks
  ranks = evaluate_performance(test_ds, 
                              model=model, 
                              filter_triples=X_filter)

  # ranks are computed per triple
  print('Test set:', X_test.shape)
  print('Size of ranks:', ranks.shape)

  # Aggregate metrics show the aggregate performance of the model on the test set using a single number
  display_aggregate_metrics(ranks)
  # ----------------------
  
  return display_aggregate_metrics(ranks)

X_train, X_val, X_test = train_set_split(dataset)
scores = training (X_train, X_val, X_test) #dataset.to_numpy()
print('........score: \n {}'.format(scores))

Total triples: (86275, 3)
Size of train: (85975, 3)
Size of valid: (100, 3)
Size of test: (200, 3)


Average TransE Loss:   0.023191:  61%|██████    | 61/100 [01:06<00:36,  1.06epoch/s]

## Knowledge Discovery

In [ ]:
from ampligraph.discovery import query_topn

def QA_retrieve (input, top_n ):
  head, relation1, tail = input[0], input[1], input[2]
  if len(relation1.split('_')) <= 2:
    relation2 = '{}_{}'.format(relation1.split('_')[1], relation1.split('_')[0])
  elif len(relation1.split('_')) == 3:
     relation2 = '{}_{}_{}'.format(relation1.split('_')[1], relation1.split('_')[2], relation1.split('_')[0])
  else:
    raise ValueError('error: relation length issue')
  # restore the previously saved model to save time
  model = restore_model('TransE.pkl')
  if relation1  in dataset['new_relation'].tolist():
    triples1, scores1 = query_topn(model, top_n=top_n, 
                                head=head, 
                                relation=relation1, 
                                tail=tail, 
                                ents_to_consider=None, 
                                rels_to_consider=None)
  else:
    triples1, scores1 = [], []

  if relation2  in dataset['new_relation'].tolist():
    triples2, scores2 = query_topn(model, top_n=top_n, 
                                head=tail, 
                                relation=relation2, 
                                tail=head, 
                                ents_to_consider=None, 
                                rels_to_consider=None)
  else:
    triples2, scores2 = [], []

  if len(triples1) + len(triples2) == top_n:
    if len(triples1) ==0: 
      triples = triples1 + triples2.tolist()
      scores = scores1+ scores2.tolist()
    else:
      triples = triples1.tolist() + triples2
      scores = scores1.tolist() + scores2
  else:
    end_idx = round(top_n/2)
    triples = triples1.tolist()[:end_idx] + triples2.tolist()[:end_idx]
    scores = scores1.tolist()[:end_idx] + scores2.tolist()[:end_idx]
   
  for triple, score in zip(triples, scores):
      print('Score: {} \t {} '.format(score, triple))
  return triples, scores
 


In [ ]:


# Query1: What are the genes/proteins related to Covid-19
print ('------------------------------------------------------------')
print('What are the genes/proteins related to Covid-19')
question3 = QA_retrieve (['covid-19', 'disease_gene', None], top_n = 20)

# Query2: What species transmit the covid-19
print ('------------------------------------------------------------')
print('What species transmit the covid-19?')
question4 = QA_retrieve (['covid-19', 'disease_species', None], top_n = 20)

# Query3: what are symptoms of the covid-19
print ('------------------------------------------------------------')
print('what are symptoms of the covid-19?')
question5 = QA_retrieve (['covid-19', 'disease_symptom', None], top_n = 20)

# Query4: what are the possible therapeutic procedure of covid-19??
print ('------------------------------------------------------------')
print('what are the possible therapeutic procedure of covid-19?')
question1 = QA_retrieve (['covid-19', 'disease_treat_procedure', None], top_n = 20)

# Query5: What are potential drugs to treat COVID-19?
print ('------------------------------------------------------------')
print('What are potential drugs to treat COVID-19?')
question6 = QA_retrieve (['covid-19', 'disease_drug', None], top_n = 20)

# Query6: What are related diseases of Covid-19
print ('------------------------------------------------------------')
print('What are related diseases of Covid-19')
question2 = QA_retrieve (['covid-19', 'disease_disease', None], top_n = 20)


# Train model, and Knowledge mining with different % of dataset

In [ ]:
ALL = []

for idx in range(len(subsets)):
  outputs = {} 
  outputs['subset_%s' %(idx+1)] = []
  data = subsets[:idx+1]
  data_concat = pd.concat(data)
  print ('i: {} -- len: {}'.format(idx, len(data_concat)))

  # Spliting dataset for training
  X_train, X_val, X_test = train_set_split(data_concat)
  # Training...
  print('Training..........')
  training (X_train, X_val, X_test)

  # Retrieving
  # Query1: What are the genes/proteins related to Covid-19
  print ('------------------------------------------------------------')
  print('What are the genes/proteins related to Covid-19')
  question3 = QA_retrieve (['covid-19', 'disease_gene', None], top_n = 20)
  df3 = pd.DataFrame(zip(question3[0],question3[1]), columns = ['triple', 'score'])
  outputs['subset_%s' %(idx+1)].append(df3)

  # Query2: What species transmit the covid-19
  print ('------------------------------------------------------------')
  print('What species transmit the covid-19?')
  question4 = QA_retrieve (['covid-19', 'disease_species', None], top_n = 20)
  df4 = pd.DataFrame(zip(question4[0],question4[1]), columns = ['triple', 'score'])
  outputs['subset_%s' %(idx+1)].append(df4)

  # Query3: what are symptoms of the covid-19
  print ('------------------------------------------------------------')
  print('what are symptoms of the covid-19?')
  question5 = QA_retrieve (['covid-19', 'disease_symptom', None], top_n = 20)
  df5 = pd.DataFrame(zip(question5[0],question5[1]), columns = ['triple', 'score'])
  outputs['subset_%s' %(idx+1)].append(df5)

  # Query4: what are the possible therapeutic procedure of covid-19??
  print ('------------------------------------------------------------')
  print('what are the possible therapeutic procedure of covid-19?')
  question1 = QA_retrieve (['covid-19', 'disease_treat_procedure', None], top_n = 20)
  df1 = pd.DataFrame(zip(question1[0],question1[1]), columns = ['triple', 'score'])
  outputs['subset_%s' %(idx+1)].append(df1)

  #Query5: What are potential drugs to treat COVID-19?
  print ('------------------------------------------------------------')
  print('What are potential drugs to treat COVID-19?')
  question6 = QA_retrieve (['covid-19', 'disease_drug', None], top_n = 20)
  df6 = pd.DataFrame(zip(question6[0],question6[1]), columns = ['triple', 'score'])
  outputs['subset_%s' %(idx+1)].append(df6)

  # Query6: What are related diseases of Covid-19
  print ('------------------------------------------------------------')
  print('What are related diseases of Covid-19')
  question2 = QA_retrieve (['covid-19', 'disease_disease', None], top_n = 20)
  df2 = pd.DataFrame(zip(question2[0],question2[1]), columns = ['triple', 'score'])
  outputs['subset_%s' %(idx+1)].append(df2)

  ALL.append(outputs)
                     
                     

In [ ]:
ALL

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
dir_path = '/content/drive/MyDrive/KG-output-2'
isExist = os.path.exists(dir_path)

if not isExist:
  os.makedirs(dir_path)

for i in range(len(ALL)): 
  print('i----------%s'%i)
  dir_s_path = '/content/drive/MyDrive/KG-output-2/subset_%s'%str(i+1)
  isExist_s = os.path.exists(dir_s_path)
  if not isExist_s:
    dir = os.makedirs(dir_s_path)
  for j  in range(len(list(ALL[i].values())[0])):
    print('j----------%s'%j)
    path = '/content/drive/MyDrive/KG-output-2/subset_%s'%str(i+1) +'/query_' + str(j) + '.csv'
    print(path)
    print((list(ALL[i].values()))[0][j])
    with open(path, 'w') as file:
      (list(ALL[i].values()))[0][j].to_csv(file)



# QA EVALUATION

In [ ]:
# Load groundtruth file
